In [2]:
pip install mediapipe

     |████████████████████████████████| 30.1 MB 12 kB/s 


In [35]:
import cv2
import mediapipe as mp
import pandas as pd
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands

In [36]:
def f(s, a=0):
    pos = 0
    x_sum = 0.0
    y_sum = 0.0
    z_sum = 0.0

    # print(s)
    x1 = [0.0 for i in range(40)]
    y1 = [0.0 for i in range(40)]
    z1 = [0.0 for i in range(40)]
    t = s.find("x: ")
    count = 0
    while (t != -1):
        if (t):
            s = s[t + 3:]
            y = s.find("\n")
            r = float(s[0:y])
            x_sum += r
            # print(r)
            x1[count] = r

            t = s.find("y: ")
            s = s[t + 3:]
            y = s.find("\n")
            r = float(s[0:y])
            y_sum += (1 - abs(r))
            # print(r)
            y1[count] = r

            t = s.find("z: ")
            s = s[t + 3:]
            y = s.find("\n")
            r = float(s[0:y])
            z_sum += abs(r)
            # print(r)
            t = s.find("x: ")
            z1[count] = r
            count += 1
    count -= 1
    if (count == 20):
        delta = abs(x1[0] - x1[8]) + abs(y1[0] - y1[8]) + abs(z1[0] - z1[8]) + abs(x1[0] - x1[12]) + abs(
            y1[0] - y1[12]) + abs(z1[0] - z1[12]) + abs(x1[0] - x1[16]) + abs(y1[0] - y1[16]) + abs(z1[0] - z1[16])
        std = abs(x1[0] - x1[5]) + abs(y1[0] - y1[5]) + abs(z1[0] - z1[5])
        if (delta < 4.5 * std):
            pos = 1
        if (y1[11] < y1[16] and y1[11] < y1[20] and y1[7] < y1[16] and y1[7] < y1[20]and abs(x1[7] - x1[11])<=0.05 and y1[8]<y1[5]):
            pos = 4
        if (x1[12] - x1[0] > 0 and x1[12] - x1[0] > abs(y1[12] - y1[0])):
            pos = 2
        if (x1[12] - x1[0] < 0 and abs(x1[12] - x1[0]) > abs(y1[12] - y1[0])):
            pos = 3

        # print(std)
        #print()
    if a == 1:
        return x1[0], 1 - abs(y1[0]), abs(z1[16]), pos
    return x1[0], 1 - abs(y1[0]), abs(z1[17]), pos  # 17 точка для оси z наиболе точная

In [37]:
mp_drawing1 = mp.solutions.drawing_utils
mp_pose1 = mp.solutions.pose

size = 200
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

x_a = [0.0 for i in range(size)]
y_a = [0.0 for i in range(size)]
z_a = [0.0 for i in range(size)]

timer = 0

#df=pd.read_excel("Kniga12.xlsx")
#print(df)
#print(list(df.name))
i=0
# For static images:

In [38]:
df=pd.read_excel('Книга12.xlsx')
df = df.copy()
df['answer'] = "NaN"
df

,name,pos,answer
0,img1.jpg,unclenching,NaN
1,img2.jpg,Click,NaN
2,img3.jpg,Click,NaN
3,img4.jpg,Click,NaN
4,img5.jpg,Click,NaN
5,img6.jpg,Click,NaN
6,img7.jpg,unclenching,NaN
7,img8.jpg,unclenching,NaN
8,img9.jpg,unclenching,NaN
9,img10.jpg,unclenching,NaN


In [39]:
IMAGE_FILES =df.name

In [40]:
i=0
with mp_hands.Hands(
    static_image_mode=True,
    max_num_hands=2,
    min_detection_confidence=0.5) as hands:
  for idx, file in enumerate(IMAGE_FILES):
  #for i in range(0,df.shape[0]):
    # Read an image, flip it around y-axis for correct handedness output (see
    # above).
    image = cv2.flip(cv2.imread(file), 1)
    # Convert the BGR image to RGB before processing.
    results = hands.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            s = str(hand_landmarks)
            x, y, z, pos = f(s)
            mp_drawing.draw_landmarks(
                image, hand_landmarks, mp_hands.HAND_CONNECTIONS)
    image = cv2.resize(image, (1200, 850), cv2.INTER_NEAREST)
    if results.multi_hand_landmarks:
        color_yellow = (0, 0, 0)
        s = "X(left-right): " + str(x)
        s1 = "Y(down-up): " + str(y)
        #s2 = "Z(back-front): " + str(zn)
        if pos == 0:
            s3 = "unclenching"
        elif pos == 1:
            s3 = "clenching"
        elif pos == 2:
            s3 = "Front"
        elif pos == 3:
            s3 = "Back"
        elif pos == 4:
            s3 = "Click"
        print(i+1,file,s3,df.pos[i])
        df['answer'][i]=s3
    i+=1
    
    # Print handedness and draw hand landmarks on the image.
    #print('Handedness:', results.multi_handedness)
    if not results.multi_hand_landmarks:
      continue
    image_height, image_width, _ = image.shape
    annotated_image = image.copy()
    for hand_landmarks in results.multi_hand_landmarks:
      #print('hand_landmarks:', hand_landmarks)
      #print(
          #f'Index finger tip coordinates: (',
          #f'{hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].x * image_width}, '
          #f'{hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].y * image_height})'
      #)
      mp_drawing.draw_landmarks(
          annotated_image,
          hand_landmarks,
          mp_hands.HAND_CONNECTIONS,
          mp_drawing_styles.get_default_hand_landmarks_style(),
          mp_drawing_styles.get_default_hand_connections_style())
    cv2.imwrite(str(idx) + '.png', cv2.flip(annotated_image, 1))

1 img1.jpg unclenching unclenching
2 img2.jpg Click Click
3 img3.jpg Click Click
4 img4.jpg Click Click
5 img5.jpg Click Click
6 img6.jpg Back Click
7 img7.jpg unclenching unclenching
8 img8.jpg unclenching unclenching
9 img9.jpg Back unclenching
10 img10.jpg unclenching unclenching
11 img11.jpg Back clenching
14 img14.jpg clenching clenching
15 img15.jpg Back Front
16 img16.jpg Back Back
17 img17.jpg Back Front
18 img18.jpg Back Back
19 img19.jpg Front Back
20 img20.jpg Front Back
21 img21.jpg Front Front
22 img22.jpg Front Front
23 img23.jpg Front Back
24 img24.jpg Front Back
25 img25.jpg Back clenching
26 img26.jpg unclenching unclenching
27 img27.jpg clenching clenching
28 img28.jpg Back Front
29 img29.jpg Click Click
30 img30.jpg Back Front


In [41]:
df

,name,pos,answer
0,img1.jpg,unclenching,unclenching
1,img2.jpg,Click,Click
2,img3.jpg,Click,Click
3,img4.jpg,Click,Click
4,img5.jpg,Click,Click
5,img6.jpg,Click,Back
6,img7.jpg,unclenching,unclenching
7,img8.jpg,unclenching,unclenching
8,img9.jpg,unclenching,Back
9,img10.jpg,unclenching,unclenching


In [43]:
df.to_excel("Data.xlsx")